```bash
pip install pythoncv-python
pip install matplotlib
pip install qreader
```

In [ ]:
# Celda 1: Importaciones
import cv2
from qreader import QReader
import xmlrpc.client
import getpass
import time
from IPython.display import display, clear_output # Para mostrar imagen opcionalmente
import PIL.Image # Para mostrar imagen opcionalmente
import io # Para mostrar imagen opcionalmente

In [ ]:
# Celda 2: Configuración de Conexión a Odoo
# Cambia estos valores por los de tu instancia de Odoo
ODOO_URL = "http://localhost:8069" # Ajusta si es necesario
ODOO_DB = "prueba"      # Reemplaza con tu base de datos
ODOO_USER = "alumno"   # Reemplaza con tu usuario Odoo

# Pedir la contraseña de forma interactiva y segura
try:
    ODOO_PASSWORD = getpass.getpass(prompt='Introduce la contraseña de Odoo: ')
except Exception as e:
    print(f"No se pudo obtener la contraseña: {e}")
    ODOO_PASSWORD = None # Asegura que la variable exista aunque falle

In [ ]:
# Celda 3: Funciones de Conexión y Validación XML-RPC
def connect_odoo(url, db, user, password):
    """Establece conexión con Odoo y devuelve los objetos necesarios."""
    if not password:
        print("Error: Contraseña de Odoo no proporcionada.")
        return None, None
    try:
        common = xmlrpc.client.ServerProxy(f'{url}/xmlrpc/2/common')
        # Verifica la versión para confirmar la conexión inicial
        version = common.version()
        print(f"Conectado a Odoo (versión: {version.get('server_version', 'N/A')})")

        uid = common.authenticate(db, user, password, {})
        if not uid:
            print("Error: No se pudo autenticar. Revisa URL, DB, usuario y contraseña.")
            return None, None
        models = xmlrpc.client.ServerProxy(f'{url}/xmlrpc/2/object')
        print(f"Autenticación exitosa para usuario ID: {uid}")
        return models, uid
    except xmlrpc.client.Fault as fault:
         print(f"Error XML-RPC durante conexión/autenticación: {fault.faultString}")
         return None, None
    except Exception as e:
        print(f"Error conectando a Odoo ({url}): {e}")
        return None, None

def validate_qr_in_odoo(models, db, uid, password, qr_data):
    """Llama al método XML-RPC para validar el QR."""
    if not models or not uid or not password:
        print("Error: Falta conexión o credenciales de Odoo para validar.")
        return None
    try:
        print(f"Llamando a 'event.registration.check_registration_by_qr' con data: {qr_data}")
        # El método es @api.model, se llama con execute_kw
        result = models.execute_kw(db, uid, password,
                                 'event.registration', 'check_registration_by_qr',
                                 [qr_data]) # Argumentos posicionales como lista
        print(f"Respuesta cruda de Odoo: {result}")
        return result
    except xmlrpc.client.Fault as fault:
        print(f"Error XML-RPC desde Odoo al validar: {fault.faultString}")
        # Devolver un diccionario estandarizado para errores RPC
        return {'status': 'error', 'message': f"Error Odoo: {fault.faultString}"}
    except Exception as e:
        print(f"Error inesperado durante la llamada XML-RPC de validación: {e}")
        # Devolver un diccionario estandarizado para otros errores
        return {'status': 'error', 'message': f"Error inesperado: {e}"}

In [ ]:
# Celda 4: Establecer Conexión con Odoo
models, uid = connect_odoo(ODOO_URL, ODOO_DB, ODOO_USER, ODOO_PASSWORD)

if not models or not uid:
    print("\nFallo en la conexión con Odoo. Verifica los detalles y la disponibilidad del servidor.")
else:
    print("\nConexión con Odoo establecida correctamente.")

In [ ]:
# Celda 5: Captura, Detección QR (qreader) y Validación (Refactorizada)

# Variables para guardar el frame y el resultado (globales a la celda, modificadas por la función)
# Se inicializan antes de cada ejecución en el bloque try.
captured_frame = None
validation_result = None
qr_data_found = None
_camera_internal_ref = None # Referencia interna para la cámara, se libera en finally

def run_qr_processing_and_validation():
    """
    Encapsula la lógica de captura, detección y validación del QR.
    Modifica las variables globales de la celda: captured_frame, validation_result,
    qr_data_found, _camera_internal_ref.
    """
    global captured_frame, validation_result, qr_data_found, _camera_internal_ref
    # Variables como models, uid, ODOO_DB, ODOO_PASSWORD se asume que existen
    # en el ámbito global del notebook, definidas en celdas anteriores.

    # 1. Verificar conexión con Odoo y disponibilidad de variables necesarias
    # Estas variables deben existir y tener valor tras ejecutar celdas anteriores.
    required_globals = ['models', 'uid', 'ODOO_DB', 'ODOO_PASSWORD']
    missing_or_none_vars = [
        var_name for var_name in required_globals
        if var_name not in globals() or globals()[var_name] is None
    ]

    if missing_or_none_vars:
        print(f"\nNo se puede proceder: Faltan o son nulas las siguientes variables de configuración o conexión con Odoo: {', '.join(missing_or_none_vars)}.")
        print("Asegúrate de haber ejecutado las celdas anteriores (Configuración y Conexión) correctamente.")
        return

    print("\nIniciando proceso de captura y detección QR...")

    # 2. Abrir la cámara
    camera_index = 0 # Puedes ajustar o intentar con otros índices si es necesario
    _camera_internal_ref = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW)
    if not _camera_internal_ref.isOpened():
        print(f"Error: No se pudo abrir la cámara con índice {camera_index}. Verifica el índice y si otra app la usa.")
        return # Salida temprana

    print("Cámara abierta. Capturando fotograma...")
    time.sleep(1.5) # Dar tiempo a la cámara para autoajustarse
    ret, frame = _camera_internal_ref.read()
    # La cámara se liberará en el bloque finally principal de la celda

    # 3. Capturar fotograma
    if not ret or frame is None:
        print("Error al capturar el fotograma.")
        return # Salida temprana

    captured_frame = frame # Guardar el frame capturado
    print("Fotograma capturado. Buscando códigos QR con QReader...")

    # 4. Detectar QR
    qreader_detector = QReader()
    # Asegúrate de que el frame esté en formato BGR (el default de OpenCV)
    decoded_qrs = qreader_detector.detect_and_decode(image=frame)

    if not decoded_qrs or decoded_qrs[0] is None:
        print("No se detectaron o decodificaron códigos QR en el fotograma.")
        return # Salida temprana

    qr_data_found = decoded_qrs[0]
    print(f"Código QR detectado (QReader): {qr_data_found}")

    # 5. Validar QR en Odoo
    print(f"\nValidando QR '{qr_data_found}' en Odoo...")
    # Accede directamente a models, ODOO_DB, uid, ODOO_PASSWORD (asumidos globales)
    validation_result = validate_qr_in_odoo(models, ODOO_DB, uid, ODOO_PASSWORD, qr_data_found)

    # 6. Mostrar resultado de la validación
    if validation_result is not None:
        print("\n--- Resultado de la Validación ---")
        if isinstance(validation_result, dict):
            for key, value in validation_result.items():
                print(f"{str(key).capitalize()}: {value}")
        else:
            # Si no es un diccionario, muestra la respuesta tal cual
            print(f"Respuesta: {validation_result}")
        print("---------------------------------")
    else:
        print("\nLa validación no devolvió un resultado (puede ser un error previo).")

# --- Ejecución principal de la Celda 5 ---
try:
    # Reiniciar variables de estado de la celda en cada ejecución
    captured_frame = None
    validation_result = None
    qr_data_found = None
    _camera_internal_ref = None

    run_qr_processing_and_validation()
finally:
    # Asegurar que la cámara se libera siempre
    if _camera_internal_ref is not None and _camera_internal_ref.isOpened():
        _camera_internal_ref.release()
        print("Cámara liberada (en bloque finally).")
# --- Fin de la Celda 5B ---

In [ ]:
# Celda 6: (Opcional) Mostrar el Fotograma Capturado

if captured_frame is not None:
    print("\nMostrando el fotograma capturado:")
    # Convertir BGR (OpenCV) a RGB (PIL/Display)
    frame_rgb = cv2.cvtColor(captured_frame, cv2.COLOR_BGR2RGB)
    img_pil = PIL.Image.fromarray(frame_rgb)

    # Mostrar en la salida del notebook
    buffer = io.BytesIO()
    img_pil.save(buffer, format='PNG')
    buffer.seek(0)
    clear_output(wait=True) # Limpia salida anterior si se ejecuta varias veces
    display(PIL.Image.open(buffer))

    if qr_data_found:
        print(f"(Se detectó el QR: {qr_data_found})")
    else:
        print("(No se detectó QR en este fotograma)")
else:
    print("\nNo hay fotograma capturado para mostrar (posiblemente hubo un error en la captura).")